# Sentiment analysis in Python

in this notebook we will be doing some sentiment naalysis in python using two different techniques

* 1. VADER (Valence Aware directory And sEntiment Resoner) - Baf of words approach
* 2. Roberta Pretrained model from 
* 3. Huggingface Pipeline 

In [1]:
# EDA (Exploratory data analysis)

import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import NLTK

plt.style.use('ggplot')

ModuleNotFoundError: No module named 'pandas'